In [1]:
!nvidia-smi

Sun Jul 28 00:50:54 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:87:00.0 Off |                    0 |
| N/A   48C    P0    39W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
dataname = "adult"

DATA_DIR = "/projects/aieng/diffusion_bootcamp/data/tabular"

TRAIN_DATA_PATH = f"{DATA_DIR}/processed_data/{dataname}/train.csv"
TEST_DATA_PATH = f"{DATA_DIR}/processed_data/{dataname}/test.csv"
TABDDPM_DATA_PATH = f"{DATA_DIR}/synthetic_data/{dataname}/tabddpm.csv"
TABSYN_DATA_PATH = f"{DATA_DIR}/synthetic_data/{dataname}/tabsyn.csv"
INFO_PATH = f"{DATA_DIR}/processed_data/{dataname}/info.json"

# Density Estimation of Single Column and Pair-wise Correlation

Two metrics are computed in this section: column shapes and column pair trends.
We explain each in the following subsections.

## Single Column Similarity Score

Essentially, each column in a table represents a single feature which accepts various values from a certain type; e.g. numerical, categorical, datetime or boolean. This feature can be described as a random variable where the values listed under the column are its samples. As a result, the density distribution of each column can be computed and compared between the real data and the synthetic data.

<p align="center">
<img src="figures/tabsyn_column_shapes_2.png" width="1000"/>
</p>

The better the distributions match each other, the better the quality of synthetic data.

The similarity of the two distriutions can be measured via different metrics for different data types; for example, **KSComplement (Kolmogorov-Smirnov Complement)** is used for numerical features and **TVComplement (Total Variation Distance Complement)** for categorical features. The overall Column Shape Score for the whole table is the average of column shape similarity scores of all columns.

**KST (Kolmogorov-Smirnov Test)**
computes the cumulative distribution function (CDF) of a numerical random variable for real and synthetic data. Then, finds the maximum different between the two CDFs $M$. Finally, the *KSComplement* score is defined as $1 - M$ so that the higher the score, the more similar the distributions.

<p align="center">
<img src="figures/tabsyn_kst.png" width="1000"/>
</p>

**TVD (Total Variation Distance)**
computes the frequency of each category's appearance under a certain column and defines it as the said categorie's probability. Then, it computes the sum of differences of probabilities between real and synthetic data as 
$\delta(R, S) = \frac{1}{2} \sum_{\omega \in \Omega}  | R_\omega - S_\omega |$
.

Here, $\omega$ describes all the possible categories in a column, $\Omega$. Meanwhile, $R$ and $S$ refer to the real and synthetic frequencies for those categories. The *TVComplement* returns $1-TVD$ so that a higher score means higher quality.

<p align="center">
<img src="figures/tabsyn_tvd.png" width="1000"/>
</p>

## Pair-wise Correlation Score

The correlation between two random variables describes how they vary in relation to each other. The higher the score, the more the trends are alike.

<p align="center">
<img src="figures/tabsyn_column_pair_trends.png" width="900"/>
</p>

Here, we use different metrics to compute the correlation between different pairs of data types:

| Column Type | Metric |
| ----------- | ------ |
| numerical & numerical | [correlation similarity](https://docs.sdv.dev/sdmetrics/metrics/metrics-glossary/correlationsimilarity) |
| categorical & categorical | [contingency similarity](https://docs.sdv.dev/sdmetrics/metrics/metrics-glossary/contingencysimilarity) |
| numerical & categorical | discretize the numerical columns into bins, then apply contingency similarity. |

This yields a score between every pair of columns. The **Column Pair Trends** score is the average of all the scores.

In [3]:
from scripts.eval.eval_density import eval_density

shape, trend = eval_density(TABDDPM_DATA_PATH, TRAIN_DATA_PATH, INFO_PATH)
print("Shape:", shape)
print("Trend:", trend)

Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 105/105 [00:05<00:00, 17.82it/s]

Overall Score: 92.07%

Properties:
- Column Shapes: 95.1%
- Column Pair Trends: 89.04%
Shape: 0.9509535947913148
Trend: 0.8904390354536156


# $\alpha$-Precision and $\beta$-Recall 

$\alpha$-Preicison and $\beta$-Recall are generalizations of Precision and Recall metrics proposed by [Sajjadi et al.](https://proceedings.neurips.cc/paper/2018/hash/f7696a9b362ac5a51c3dc8f098b73923-Abstract.html) in 2018. These metrics can range between $[0, 1]$ and the closest they are to $1$ the better.


Preicison measures the fidelity or quality of sythetic data. In more clear terms, it computes the proporation of synthetic datapoints that are *close* to real datapoints. 
Recall, on the other hand, measures the diversity of synthetic data; i.e. the extent to which these samples cover the full variability of real samples. More clearly, recall computes the proportion of real datapoints that are *close* to synthetic datapoints.


If we denote the distibution of real datapoints by $P(X)$ and the distribution of sythetic datapoints by $Q(Y)$, precision is the portion of $Q(Y)$ that can be generated by $P(X)$, while recall is the portion of $P(X)$ that can be generated by $Q(Y)$.

To better understand these concepts, let's assume that the real/generated dataponits are samples from an underlying real/generated manifold.
Precision measures the proportation of generated datapoints that fall on the real manifold, while recall measures the proportion of real datapoints that fall on the generated manifold.

<p align="center">
<img src="figures/tabsyn_precision.png" width="300"/>
</p>

<p align="center">
<img src="figures/tabsyn_recall.png" width="300"/>
</p>

The Precision-Recall metrics are very sensitive to outliers since even a few outliers can greatly change the shape of the underlying manifold.
To address this limitation, $\pmb{\alpha}$**-Precision** and $\pmb{\beta}$**-Recall** are defined by assuming that a fraction $1−\alpha$ (or $1−\beta$) of the real (and synthetic) data are “outliers”, and $\alpha$ (or $\beta$) are “typical”. 
$\alpha$-Precision is the fraction of synthetic samples that resemble the “most typical” fraction $\alpha$ of real samples, whereas $\beta$-Recall is the fraction of real samples covered by the most typical fraction $\beta$ of synthetic samples.
The two metrics are evaluated for all $\alpha, \beta \in [0, 1]$, providing entire precision and recall curves instead of single numbers.


To illustrate, consider the below image. Blue and red points are real and generated datapoints, respectively. The large blue and red spheres show the underlying manifold that was estimated from real and generated datapoints.
Good quality generated datapoints should fall within the blue sphere like image *(c)*. They should not lie far from the blue sphere like *(a)*. Moreover, they should not be placed too close (or *copied*) to a real datapoint like *(b)*.
Image *(d)* shows an outlier in the real datapoints which is cut outside of the manifold due to the application on $\alpha$ and $\beta$.
If we used vanilla Precision and Recall, the blue sphere's radius should have increased to include the outlier which would also lead it to include noisy synthetic datapoints like *(a)*.

<p align="center">
<img src="figures/tabsyn_alpha_precision_beta_recall.png" width="500"/>
</p>


In [4]:
from scripts.eval.eval_quality import eval_quality

alpha_precision_all, beta_recall_all = eval_quality(
    TABDDPM_DATA_PATH, TRAIN_DATA_PATH, INFO_PATH
)
print("Alpha precision:", alpha_precision_all)
print("Beta recall:", beta_recall_all)

/projects/aieng/diffusion_bootcamp/data/tabular/synthetic_data/adult/tabddpm.csv
=========== All Features ===========
Data shape:  (32561, 110)
Alpha precision: 0.9516072220945514
Beta recall: 0.1481015120338237


# Machine Learning Efficiency

This method trains a machine learning model (in our case, an XGBoost model) on the synthetic data and evaluates it on the real data.

In [5]:
from scripts.eval.eval_mle import eval_mle
from pprint import pprint

## does a grid search over given params and reports all scores for each best of them
# tabular dataload and tabular transformer look extra
overall_score = eval_mle(TABDDPM_DATA_PATH, TEST_DATA_PATH, INFO_PATH)
print("Overall score:")
pprint(overall_score)

100%|██████████| 36/36 [00:16<00:00,  2.25it/s]


Overall score:
{'best_acc_scores': {'XGBClassifier': {'accuracy': 0.8360051593882439,
                                       'binary_f1': 0.6148297749567224,
                                       'roc_auc': 0.8857347233173605,
                                       'weighted_f1': 0.6812082789382777}},
 'best_auroc_scores': {'XGBClassifier': {'accuracy': 0.8384005896443707,
                                         'binary_f1': 0.6180868050515315,
                                         'roc_auc': 0.8852936256573706,
                                         'weighted_f1': 0.6840959832187936}},
 'best_avg_scores': {'XGBClassifier': {'accuracy': 0.8360051593882439,
                                       'binary_f1': 0.6148297749567224,
                                       'roc_auc': 0.8857347233173605,
                                       'weighted_f1': 0.6812082789382777}},
 'best_f1_scores': {'XGBClassifier': {'accuracy': 0.8360051593882439,
                                      'b

# Privacy Protection: Distance to Closest Record (DCR)

One of the applications of synthetically generated data is protecting sensitive information while creating similar substitute data that could be used to train machine learning models or published on public platforms.
For this purpose, we must ensure that the synthetic datapoints are far enough from any real datapoints to prevent leaking of real sensitive information.

One metric that is used for this purpose is **Distance to Closest Record (DCR)**.
DCR is the Euclidean distance between a synthetic datapoint and its nearest real datapoint.
DCR equal to zero means that the synthetic datapoint will leak the real information, while higher DCR values mean less risk of privacy leakage.

`eval_dcr` computes the DCR of each synthetic datapoint to real datapoints in two different sets: training and test. Then, it returns the proportion of synthetic datapoints that are closer to the training dataset than the test dataset.
If the size of the training and test datasets are equal, this score should ideally be $0.5$ indicating that the model has not overfit to training data and the synthetic datapoints are not memorized copies of training data.
If the size of the training and test datasets are different, the ideal value for this score is #Train / (#Train + #Test).

In [6]:
from scripts.eval.eval_dcr import eval_dcr

dcr_score = eval_dcr(TABDDPM_DATA_PATH, TRAIN_DATA_PATH, TEST_DATA_PATH, INFO_PATH)
print("DCR Score, a value closer to 0.5 is better")
print("Distance to Closest Record:", dcr_score)

DCR Score, a value closer to 0.5 is better
Distance to Closest Record: 0.673320843954424


# Detection: Classifier Two Sample Tests (C2ST)

This metric evaluates if the synthetic data can be detected from the real data via a machine learning model, hence measuring how difficult it is to distinguish synthetic from real data. A logistic regression model is used in `eval_detection`.

This score is measured through below steps:
1. Create a single, augmented table that has all the rows of real data and all the rows of synthetic data. Add an extra column to keep track of whether each original row is real or synthetic.
2. Split the augmented data to create a training and validation sets.
3. Choose a machine learning model. Train the model on the training split. The model will predict whether each row is real or synthetic (i.e. predict the extra column we created in step #1).
4. Validate the model on the validation set.
5. Repeat steps #2-4 multiple times.
The final score is based on the average ROC-AUC score across all the cross validation splits,

$score = 1 - (max($ <span style="text-decoration:overline">ROC-AUC</span> $, 0.5) \times 2 - 1)$
.

This score can range between $[0, 1]$ with $0$ being the lowest (meaning that the machine learning model can perfectly identify synthetic data apart from the real data), and $1$ being the highest (meaning that the machine learning model cannot identify the synthetic data apart from the real data).



In [7]:
from scripts.eval.eval_detection import eval_detection

detection_score = eval_detection(TABDDPM_DATA_PATH, TRAIN_DATA_PATH, INFO_PATH, dataname, model="tabddpm")
print("Detection score:", detection_score)

Detection score: 0.6736000867078136
